# TODO:
* **Criar uma funcao que colete os links de todos os imoveis a venda da imobiliaria ✓**

* **Criar um dicionario com as informacoes de: Preco ✓**

# Estudo da página

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [9]:
driver = webdriver.Firefox(executable_path='/home/douglas/PycharmProjects/scraperImperium/geckodriver')

/tmp/ipykernel_3897/1040251504.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='/home/douglas/PycharmProjects/scraperImperium/geckodriver')


In [10]:
driver.get('https://www.imperiumimoveis.com.br/imoveis/venda-guarapuava')

**Obtendo quantidade de imóveis**

In [4]:
qtd = driver.find_element(By.CLASS_NAME, 'list__counter').text
qtd = int(qtd.split()[0])

**O tamanho inicial já conta com 12 anúncios. Cada seção/load da página adiciona 3960 pixels/12 anúncios. Existem 413 anúncios**

In [5]:
def rolar_pag():
    # Impede que o site "emperre"
    driver.execute_script('window.scrollBy(0, -500)')
    # Elemento que faz com que o site carregue mais anúncios
    elemento = driver.find_element(By.XPATH, '/html/body/main/div/section/div[2]/div[1]/div[3]/div[2]')
    #Move para o elemento que carrega a página
    driver.execute_script("arguments[0].scrollIntoView();", elemento)

**Carregando todos os anúncios**

In [7]:
maximo = 0
while True:
    rolar_pag()
    sleep(1.5)
    aux = driver.execute_script('return document.body.scrollHeight')
    if maximo < aux:
        maximo = aux
    else:
        break

O elemento ```<div class="jetgrid list__container" data-template="">``` é pai de todos os elementos de anúncios (```<div class="jetgrid__col--24 jetgrid__col--sm-12 jetgrid__col--xs-24">```)

XPath pai:
```/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]```

XPath 1º anúncio:
```/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[1]```

XPath 2º anúncio:
```/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[2]```

# Coleta dos demais dados

In [15]:
def get_referencia(i:int) -> str:
    return driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a/div[2]/p[1]').text

In [20]:
def get_area_t(i:int) -> float:
    area = driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a/div[2]/ul/li[1]/span').text
    area = float(area.replace('M² TOTAL', ''))
    return area

In [27]:
def get_area_u(i:int) -> float:  # Nem todos os anúncios tem
    area = driver.find_element(By.XPATH, f'/html/body/main/div/section/div[2]/div[1]/div[3]/div[1]/div[{i}]/a/div[2]/ul/li[2]/span').text
    if 'M² ÚTIL' in area:
        area = float(area.replace('M² ÚTIL', ''))
        return area
    return 0

In [28]:
area_t = dict()
area_u = dict()

for i in range(1, 12+1):
    ref = get_referencia(i)
    area_t[ref] = get_area_t(i)

    area_u[ref] = get_area_u(i)

print('area_t: ', area_t)
print('area_u: ', area_u)

area_t:  {'09970.5308': 20000.0, '09970.4672': 17147.19, '09970.4130': 1972.32, '09970.4786': 2100.0, '09970.4321': 646.76, '01587.019': 252.0, '09970.5348': 825.0, '09970.4132': 22000.0, '09970.4659': 1100.0, '09970.4131': 580.0, '09970.4352': 500.0, '09970.4718': 516.26}
area_u:  {'09970.5308': 3351.0, '09970.4672': 20000.0, '09970.4130': 600.0, '09970.4786': 2100.0, '09970.4321': 646.76, '01587.019': 0, '09970.5348': 500.0, '09970.4132': 0, '09970.4659': 500.0, '09970.4131': 800.0, '09970.4352': 0, '09970.4718': 579.85}
